In [1]:
import pymongo
import os,csv,sys
import openslide
import dsa_mongo_common_functions as dsa
import cdsa_loader_helper_functions as cdsa_helpers
import pprint
client = pymongo.MongoClient('localhost',27017)
dsa_load_errors_db = client['DSA_LoadErrors']


In [2]:
reload(dsa)
reload(cdsa_helpers)


<module 'cdsa_loader_helper_functions' from 'cdsa_loader_helper_functions.pyc'>

In [3]:
## This is specific to a given fle system and or structurme
slide_root = '/GLOBAL_SCRATCH/ADRC/'  ##Base Path for Slides
dsa_slide_db = client['DSA_Slide_DB']  ### These need to be configured for the specific project

### To generalize this, need to describe organization, most common will be  PATIENT/STAIN_TYPE as subdirectories
subj_dir_list = [x for x in os.listdir(slide_root) if os.path.isdir(os.path.join(slide_root,x))]
print len(subj_dir_list),"Potential Patient directories were identified"

103 Potential Patient directories were identified


In [4]:
def find_rawslide_lists( slide_root_path ):
        """project_name is passed along with the potentially more than one root image path for ndpi files"""
        slide_files = []

        slide_root_path  = slide_root_path.rstrip('/')
        print slide_root_path
        for dpath, dnames, fnames in os.walk( slide_root_path, followlinks=True):
                
                for file in fnames:
                    if '.ndpi' in file or '.svs' in file:
                                slide_files.append(dpath +'/'+file)
        print len(slide_files),"SVS or NDPI files were located"
        return slide_files

In [5]:
print subj_dir_list

['ADRC61-124', 'ADRC50-10', 'ADRC61-128', 'ADRC50-40', 'ADRC51-60', '@eaDir', 'ADRC60-150', 'ADRC61-120', 'ADRC57-85', 'ADRC55-200', 'ADRC49-07', 'ADRC50-11', 'ADRC51-2', 'ADRC51-134', '.ipynb_checkpoints', 'SamMarch20', 'ADRC61-139', 'ADRC61-125', 'ADRC53-300', 'ADRC56-112', 'ADRC43-24', 'ADRC61-73', 'ADRC62-74', 'ADRC61-84', 'ADRC62-14', 'ADRC60-179', 'ADRC49-04', 'ADRC56-97', 'ADRC51-06', 'ADRC36-04', 'ADRC59-164', 'ADRC56-18', 'ADRC59-81', 'ADRC62-110', 'ADRC61-143', 'ADRC47-60', 'ADRC62-6', 'ADRC60-48', 'ADRC61-97', 'ADRC62-64', 'ADRC54-125', 'newcerebptstoload', 'ADRC61-71', 'ADRC60-110', 'ADRC61-20', 'ADRC56-06', 'ADRC53-55', 'ADRC59-8', 'ADRC60-137', 'ADRC60-160', 'ADRC61-70', 'ADRC61-75', 'ADRC55-100', 'ADRC56-160', 'ADRC51-10', 'ADRC39-44', 'ADRC58-146', 'ADRC60-88', 'ADRC61-41', 'ADRC62-62', 'ADRC61-112', 'ADRC55-194', 'ADRC60-83', 'ADRC50-41', 'ADRC60-38', 'ADRC61-60', 'ADRC62-32', 'ADRC54-76', 'NickSlidesFeb20', 'ADRC59-35', 'ADRC47-61', 'ADRC55-108', 'ADRC61-83', 'ADRC51-

In [6]:
import hashlib
def generate_file_md5(rootdir, filename, blocksize=2**20):
    m = hashlib.md5()
    with open( os.path.join(rootdir, filename) , "rb" ) as f:
        while True:
            buf = f.read(blocksize)
            if not buf:
                break
            m.update( buf )
    return m.hexdigest()

In [8]:
# nm = '/GLOBAL_SCRATCH/ADRC/ADRC53-300/Tau/ADRC53-300_1_TAU.ndpi'
# generate_file_md5('/GLOBAL_SCRATCH/ADRC/ADRC53-300/Tau/','ADRC53-300_1_TAU.ndpi')

NameError: global name 'hashlib' is not defined

In [ ]:
### So this creates a document WITHIN the current database to store raw information about the slides
## Now that I think about this, I really should not store the filename but the File Hash as I.. want to change the filenames

for sd in subj_dir_list:
    print sd,"is being processed",
    curr_svs_slide_list = find_rawslide_lists(  os.path.join(slide_root,sd)  )
    slides_processed = newly_processed = dup_slide = rescanned_slides =  0
    for sld in curr_svs_slide_list:

        slide_name = os.path.basename(sld)
        qry = dsa_slide_db['RawSlideData'].find_one( {'slide_name':slide_name})
        fs = os.path.getsize(sld)
        ### Going to add the filesize... so I make the assumption if the name and the filesize are the same,
        ### I have the same file..
        #print qry
        if not qry:
            
            #md5Checksum = dsa.md5sum(sld)
            (openslide_could_open, width, height, filesize, orig_resolution, slide_name,md5, sld_properties) = cdsa_helpers.openslide_test_file_mongo( sld, 'ndpi', client)
            if openslide_could_open:
                prep_type = 'Unknown'
                slide_metadata = { 'slide_w_path': sld, 'slide_name': slide_name, 'file_size':fs, 'width':width, 'height':height,
                                 'orig_resolution': orig_resolution, 'sld_properties': cdsa_helpers.clean_openslide_keys ( sld_properties), 'slide_md5': md5, 'prep_type': prep_type
                                 }
                dsa_slide_db['RawSlideData'].insert_one(slide_metadata)
                newly_processed +=1 
            else:
                print "UNABLE TO OPEN FILE??",sld
                ###Need to flag/load this in to an error database

        else:
            fs = os.path.getsize(sld)
            ## Double check if file size matches
#             if qry['file_size'] != fs:
#                 #print "File size mismatch??",fs,qry['file_size'],qry['slide_w_path'],sld
#                 load_errors_db['rescanned_slides'].insert_one( {'loaded_slide': qry['slide_w_path'], 'rescanned_slide': sld}             )
#                 rescanned_slides +=1 
#             else:
#                 dup_slide +=1

    
        slides_processed +=1        
        output = "Total Processed: %d  Newly Processed: %d Dup Slides or Already Loaded: %d  RESCANNED Slides %d" % (slides_processed, newly_processed, dup_slide, rescanned_slides )
        dsa.LinePrinter(output)


ADRC61-124 is being processed /GLOBAL_SCRATCH/ADRC/ADRC61-124
106 SVS or NDPI files were located
Total Processed: 44  Newly Processed: 44 Dup Slides or Already Loaded: 0  RESCANNED Slides 0

In [ ]:
#dsa_slide_db['DSA_Slide_Data'].delete_many({})
#print dsa_slide_db['DSA_Slide_Data'].count()

In [ ]:
### I now want to reformat all of this data to make it more useful for DSA ...

In [ ]:
all_slides = dsa_slide_db['RawSlideData'].find()

## Since this is pretty easy to recreate, I'm going to empty the current database

dsa_slide_db['DSA_Slide_Data'].delete_many({})

for s in all_slides:
    slide_dict = {}
    pt_id = s['slide_w_path'].split('/')[-3]
    stain_type = s['slide_w_path'].split('/')[-2]
    slide_dict = s
    slide_dict.pop('_id',None)
    
    slide_dict['pt_id'] = pt_id
    slide_dict['stain_type'] = stain_type
    ### Obfuscating the global file path so everything is relative to some base path for the archive/
    slide_dict['thumbnail_image'] = '/thumbnail/' + s['slide_w_path'].replace('/GLOBAL_SCRATCH/ADRC/','')
    slide_dict['slide_w_path'] = '/DZIMS/' + s['slide_w_path'].replace('/GLOBAL_SCRATCH/ADRC/','')+'.dzi'
    dsa_slide_db['DSA_Slide_Data'].insert_one(slide_dict)

    #       var html = sld_info.slide_name + '#' + sld_info.slide_w_path+ '#' + sld_info.slide_name + '#' + sld_info.thumbnail_image;
##    viewer.open('/DZIMS/ADRC50-21/pTDP/OS00-21_14_pTDP_1to10k.ndpi.dzi')
###      var html = sld_info.slide_name + '#' + sld_info.slide_w_path+ '#' + sld_info.slide_name + '#' + sld_info.thumbnail_image;


In [ ]:
print dsa_slide_db['DSA_Slide_Data'].distinct('pt_id')

In [ ]:
cur= dsa_slide_db['DSA_Slide_Data'].find({'pt_id':'ADRC50-10'})
for c in cur:
    print c

In [ ]:

## Going to create a cleanup and /or reformatted collection for the DSA Viewer
for s in dsa_slide_db['ADRC'].find():
    keys_of_interest = ['width','height']
    print s['slide_w_path']
    print s.keys()
    sys.exit()
###
